In [5]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_ollama import OllamaLLM
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
import os

# Setup memory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history"
)

# Setup prompt with a placeholder for the chat history
prompt = ChatPromptTemplate(
    input_variables=["input"],
    messages=[
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}")
    ]
)

# Configure Ollama host and port
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "127.0.0.1")
OLLAMA_PORT = os.getenv("OLLAMA_PORT", "11434")
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

# Initialize Ollama LLM
chat = OllamaLLM(model="llama3.2:3b", base_url=OLLAMA_URL, timeout=30)

# Create a runnable chain
chain = {"input": RunnablePassthrough()} | prompt | chat

# Wrap the chain with message history
conversational_chain = RunnableWithMessageHistory(
    chain,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="history",
    output_messages_key="response"
)

# Session management (simple session ID for this example)
session_id = "default_session"

while True:
    content = input(">> ")
    if content.lower() == 'bye':
        break
    result = conversational_chain.invoke(
        {"input": content},
        config={"configurable": {"session_id": session_id}}
    )

    # Handle potential key changes in output
    if "response" in result:
        print(result["response"])
    else:
        print(result.get("text", "No response available"))  # Fallback for different output keys

AttributeError: 'ConversationBufferMemory' object has no attribute 'messages'

In [19]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_ollama import OllamaLLM
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
import os
import re

# Define the state structure
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Setup memory with MemorySaver
memory = MemorySaver()

# Setup prompt with a placeholder for the chat history and instruction
prompt = ChatPromptTemplate(
    input_variables=["input"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("Please provide a direct answer. {input}")
    ]
)

# Configure Ollama host and port
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "127.0.0.1")
OLLAMA_PORT = os.getenv("OLLAMA_PORT", "11434")
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

# Initialize Ollama LLM
chat = OllamaLLM(model="llama3.2:3b", base_url=OLLAMA_URL, timeout=30)

# Define the node function with explicit state handling and context awareness
def call_llm(state):
    # Get existing messages with fallback
    messages = state.get("messages", [])
    print(f"Debug: State messages = {messages}")  # Debug output
    # Use the latest user message as input, fallback to empty if none
    user_input = messages[-1].content if messages and messages[-1].type == "human" else ""
    if not user_input:
        raise ValueError("No valid user message in state")
    
    # Extract the last result if available for follow-up commands
    last_result = messages[-2].content if len(messages) > 1 and messages[-2].type == "ai" else ""
    if "add" in user_input.lower() and last_result:
        try:
            # Use regex to extract the first number from the last result
            match = re.search(r'\b(\d+\.?\d*)\b', last_result)  # Match any standalone number
            if match:
                last_number = float(match.group(1))  # Extract the first number
                # Extract the number to add from user_input (look for the number after "add")
                add_match = re.search(r'add\s+(\d+\.?\d*)', user_input.lower())
                if add_match:
                    add_value = float(add_match.group(1))  # Extract the number after "add"
                    response_content = f"{last_number} + {add_value} = {last_number + add_value}"
                else:
                    response_content = f"Sorry, I couldn't find a valid number to add in: {user_input}"
            else:
                response_content = f"Sorry, I couldn't find a valid number in the last result: {last_result}"
        except ValueError:
            response_content = f"Sorry, I couldn't perform the calculation based on {last_result}."
    else:
        # Format the prompt with the full message history and new input
        prompt_messages = prompt.format_messages(messages=messages, input=user_input)
        try:
            response = chat.invoke(prompt_messages)
            response_content = response if isinstance(response, str) else response.content
        except Exception as e:
            print(f"LLM Invocation Error: {e}")
            return {"messages": messages}  # Return unchanged state on error
    
    return {"messages": messages + [AIMessage(content=response_content)]}

# Build the graph
workflow = StateGraph(State)
workflow.add_node("llm", call_llm)
workflow.set_entry_point("llm")
workflow.add_edge("llm", END)

# Compile the graph with memory
app = workflow.compile(checkpointer=memory)

# Thread (session) management
thread_id = "thread_1"

while True:
    content = input(">> ")
    if content.lower() == 'bye':
        break
    # Initialize state with the new user message
    initial_state = {"messages": [HumanMessage(content=content)]}
    # Invoke the graph with the thread_id for memory persistence
    try:
        result = app.invoke(initial_state, config={"configurable": {"thread_id": thread_id}})
        # Print the latest AI response
        if result["messages"]:
            ai_response = result["messages"][-1].content
            print(ai_response)
        else:
            print("No response available")
    except Exception as e:
        print(f"Error: {e}")

Debug: State messages = [HumanMessage(content='1+1=?', additional_kwargs={}, response_metadata={}, id='8dc251bc-a8e8-40ca-b1f3-2267ddc921f3')]
2.
Debug: State messages = [HumanMessage(content='1+1=?', additional_kwargs={}, response_metadata={}, id='8dc251bc-a8e8-40ca-b1f3-2267ddc921f3'), AIMessage(content='2.', additional_kwargs={}, response_metadata={}, id='19afd928-1273-4535-aa7f-a369c33a289e'), HumanMessage(content='add 3 to the result', additional_kwargs={}, response_metadata={}, id='10be72b2-cb2b-4233-83ee-34cf278c8b0a')]
2.0 + 3.0 = 5.0
